In [1]:
import d2ltvm
import tvm
from tvm import te
import numpy as np

successd...



In [2]:
def matmul(n, m, l):
    """ Return the computing expression of matrix multiplication
        A : n x l matrix
        B : l x m matrix
        C : n x m matrix with C = A B
    """
    A = te.placeholder((n, l), name='A', dtype='float32')
    B = te.placeholder((l, m), name='B', dtype='float32')
    k = te.reduce_axis((0, l), name='k')
    C = te.compute((n, m), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name='C')
    return A, B, C

In [3]:
n = 100
A, B, C = matmul(n, n, n)
s = te.create_schedule(C.op)
tvm.lower(s, [A, B, C]).show()

In [7]:
mod = tvm.build(s, [A, B, C])
a, b, c = d2ltvm.get_abc((n, n), tvm.nd.array)
mod(a, b, c)
np.testing.assert_allclose(np.dot(a.asnumpy(), b.asnumpy()), c.asnumpy(), atol=1e-5)

# Summary
1.We can express the computation of matrix multiplication in TVM in one line of code.

2.The naive matrix multiplication is a 3-level nested for loop.